# Plastizität: Die Kunst der Anpassung

Plastizität beschreibt die Fähigkeit von Erinnerungen und neuronalen Verbindungen, sich zu verändern: Sie passen sich neuen Erfahrungen an, verstärken sich, verblassen oder verschmelzen. Diese Eigenschaft ermöglicht es biologischen Systemen, aus Erfahrungen zu lernen und sich flexibel an veränderte Umgebungen anzupassen.

Die neurobiologischen Grundlagen der Plastizität wurden durch Donald Hebb in seinem berühmten Postulat beschrieben:

*„When an axon of cell A is near enough to excite cell B and repeatedly or persistently takes part in firing it, some growth process or metabolic change takes place such that A's efficiency as one of the cells firing B is increased."* 

Hebb betonte dabei Kausalität und zeitliche Präzedenz: Zelle A muss zur Aktivierung von Zelle B beitragen – nicht nur gleichzeitig aktiv sein. Die populäre Vereinfachung *„Cells that fire together, wire together."* (geprägt von Carla Shatz) erfasst diese kausale Beziehung nicht vollständig. 

Hebbian Learning manifestiert sich in zwei komplementären Prozessen: **Langzeitpotenzierung (LTP)**, die häufig genutzte synaptische Verbindungen stärkt, und **Langzeitdepression (LTD)**, die ungenutzte Verbindungen schwächt – das Prinzip *„Use it or lose it"*.

Diese Mechanismen haben unmittelbaren Einfluss auf die Entwicklung künstlicher neuronaler Netze. Die wiederholte Präsentation von Trainingsdaten über mehrere Epochen entspricht der synaptischen Verstärkung durch wiederholte Aktivierung. Je öfter ein neuronales Netz bestimmte Muster sieht, desto stärker werden die entsprechenden Gewichte angepasst – eine direkte Analogie zur biologischen Plastizität.

In Multi-Agent-Systemen lässt sich Plastizität durch adaptive Gedächtnisarchitekturen umsetzen, die sich kontinuierlich reorganisieren. Dabei umfasst sie sowohl **strukturelle Plastizität** (Bildung und Auflösung von Verbindungen) als auch **funktionale Plastizität** (Anpassung bestehender Gewichte).

---

## Synaptische Plastizität – Neurobiologische Grundlagen und praktische Anwendung

### Problemstellung und Motivation

Finanz- und Versicherungsunternehmen agieren in einer Umgebung, in der sich Betrugsmuster, Kundenerwartungen und regulatorische Rahmenbedingungen kontinuierlich verändern. Klassische regelbasierte Frühwarnsysteme sind auf historische Wissensstände beschränkt und verlieren rasch an Wirksamkeit, sobald Angreifer neue Taktiken anwenden oder Kundinnen und Kunden ihr Verhalten anpassen. Zudem führen Fehlalarme zu erhöhter operativer Belastung und beeinträchtigen das Kundenerlebnis.

Die Neurobiologie zeigt mit der **synaptischen Plastizität**, dass lernfähige Systeme ihre Verbindungsstärken dynamisch anpassen können. **Langzeitpotenzierung (LTP)** verstärkt erfolgreiche Signalwege, während **Langzeitdepression (LTD)** ineffektive Verbindungen abschwächt. Diese Mechanismen gewährleisten, dass das Gehirn neue Muster integriert, ohne zuvor Gelerntes vollständig zu überschreiben (catastrophic forgetting). Überträgt man diese Prinzipien auf analytische Plattformen, entsteht ein adaptives Gedächtnis, das mathematisch nachvollziehbar auf Erfahrungswissen reagiert.

Im Kontext der Betrugserkennung ermöglicht plastisches Lernen, feingranulare Wechselwirkungen zwischen Transaktionsmerkmalen zu bewerten und laufend zu aktualisieren. Zugleich lassen sich regulatorische Anforderungen an **Erklärbarkeit** und **Nachvollziehbarkeit** (Explainable AI, XAI) erfüllen.

**Neurobiologischer Referenzpunkt:**  
Mechanismen der synaptischen Plastizität (LTP, LTD) als biologische Grundlage adaptiver Informationsverarbeitung.

### Praktische Relevanz im Finanzwesen

Betrugserkennung, Kreditrisikomanagement und Compliance-Überwachung profitieren von kontinuierlich lernenden Modellen, die flexibel auf neue Muster reagieren und regulatorisch prüfbar bleiben.

Die biologischen Prinzipien lassen sich auf analytische Plattformen übertragen und in drei Leitlinien verdichten:

- **Koinzidenzdetektion als Merkmalsinteraktion:** Plastische Systeme bewerten nicht isolierte Features, sondern deren gemeinsame Aktivierung. In der Praxis bedeutet dies, Gewichtsmatrizen über Feature-Kombinationen zu führen.  
- **Kalziumdynamik als adaptive Lernrate:** Leaky-Integrator-Modelle für Kalzium implementieren schwellenbasierte Regulation: Niedrige Ca²⁺-Level begünstigen LTD (Phosphatasen), hohe Ca²⁺-Level begünstigen LTP (Kinasen). Dies sichert Stabilität und vermeidet Überanpassung.  
- **Schwellensteuerung als Governance:** Adaptive Schwellenwerte werden an Zielmetriken (z. B. gewünschte False-Alarm-Rate) ausgerichtet und schaffen auditierbare Eingriffspunkte für Risk- und Compliance-Teams.

Diese methodischen Leitplanken bilden die Grundlage für die anschließende Fallstudie im Finanzwesen.

---

## Setup und Imports

In [1]:
from __future__ import annotations

import importlib.util
import sys
import warnings
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

warnings.filterwarnings("ignore")

# Notebook-Styling laden
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = (
    NOTEBOOK_DIR.parent
    if (NOTEBOOK_DIR.parent / "notebook_style.py").exists()
    else NOTEBOOK_DIR
)

spec = importlib.util.spec_from_file_location(
    "notebook_style", PROJECT_ROOT / "notebook_style.py"
)
if spec is None or spec.loader is None:
    raise ImportError

nb_style = importlib.util.module_from_spec(spec)
sys.modules["notebook_style"] = nb_style
spec.loader.exec_module(nb_style)

PLOT_COLORS = nb_style.setup_plot_style(
    aliases={
        'prediction': 'primary',
        'actual': 'secondary',
        'weight': 'accent',
        'error': 'quaternary',
    },
    cycle_keys=("primary", "secondary", "accent"),
)

SEED = int(getattr(nb_style, 'SEED', 42))
np.random.seed(SEED)

---

### LTP/LTD

#### *Langzeitpotenzierung (LTP) – Verstärkung*
Wenn ein Feature **koaktiv** mit einem **positiven Fehler** ist, wird die Verbindung stärker.

#### *Langzeitdepression (LTD) – Abschwächung*
Wenn ein Feature **koaktiv** mit einem **negativen Fehler** ist, wird die Verbindung schwächer.

#### *Die Delta-Regel (Error-modulierte Hebb'sche Regel)*

**Vorhersage:**
$$\hat{y} = \sigma(w \cdot x)$$

**Gewichtsupdate:**
$$\Delta w = \eta (y - \hat{y}) x - \alpha w$$

**Interpretation:**
- $\hat{y}$ = Vorhersage (geschätztes Risiko)
- $w$ = Gewichte (Synapsenstärken)
- $x$ = Features (Eingaben)
- $\eta$ = Lernrate (wie schnell wir lernen)
- $(y - \hat{y})$ = Fehler (Fehler-Signal)
- $\alpha$ = Decay-Rate (Homeostase, verhindert Überanpassung)
- $\sigma$ = Sigmoid-Aktivierungsfunktion

**Neurobiologisches Mapping:**
- **LTP:** Wenn $y > \hat{y}$ (positiver Fehler) -> Dopamin-Signal -> Kinasen aktiv -> Gewicht steigt
- **LTD:** Wenn $y < \hat{y}$ (negativer Fehler) -> Phosphatasen aktiv -> Gewicht sinkt
- **Homeostase:** $-\alpha w$ verhindert ungebremstes Wachstum (Homeostatic Scaling)

---

### Praktisches Szenario: Betrugserkennung

**Problem:** Eine Bank möchte Betrug erkennen. Aber Betrüger ändern ständig ihre Taktiken.

**Lösung:** Ein adaptives System, das von Fehlern lernt.

**Features (4 Beispiele):**
- $x_1$ = "neues Gerät" (0 oder 1)
- $x_2$ = "Nacht" (0 oder 1)
- $x_3$ = "Ausland" (0 oder 1)
- $x_4$ = "hoher Betrag" (0 oder 1)

**Label:**
- $y$ = 1 (Betrug) oder 0 (legitim)

**Streaming mit Drift (Regimewechsel):**
- Phase 1 (t=0-100): Betrugsbasisrate = 5%
- Phase 2 (t=100-200): Betrugsbasisrate = 15% (Betrüger ändern Taktik!)
- Phase 3 (t=200-300): Betrugsbasisrate = 8%

Das System muss sich an diese Veränderungen anpassen – genau wie das Gehirn.

### Adaptive Gewichte

Statt fester Regeln verwenden wir **adaptive Gewichte**, die sich an neue Informationen anpassen:

$$\Delta w = \eta (y - \hat{y}) x - \alpha w$$

**Was passiert?**

1. **Vorhersage:** Wir schätzen das Risiko: $\hat{y} = \sigma(w \cdot x)$

2. **Fehler berechnen:** Wie falsch war unsere Vorhersage? $error = y - \hat{y}$

3. **LTP (Verstärkung):** Wenn $error > 0$ (wir unterschätzt haben):
   - Koaktivierte Features werden verstärkt
   - Gewichte steigen
   - Nächstes Mal werden wir vorsichtiger

4. **LTD (Abschwächung):** Wenn $error < 0$ (wir überschätzt haben):
   - Koaktivierte Features werden abgeschwächt
   - Gewichte sinken
   - Nächstes Mal werden wir weniger vorsichtig

5. **Homeostase:** Der Term $-\alpha w$ verhindert, dass Gewichte unbegrenzt wachsen
   - Erhält die Lernfähigkeit des Systems
   - Biologisch: Homeostatic Scaling

**Neurobiologisches Mapping:**
- Fehler-Signal $(y - \hat{y})$ <-> Dopamin-Signal
- Koaktivität $(x, error)$ <-> LTP/LTD
- Homeostase $-\alpha w$ <-> Rezeptor-Normalisierung

---

### Implementierung

In [2]:
def sigmoid(x):
    """Sigmoid-Aktivierungsfunktion."""
    return 1.0 / (1.0 + np.exp(-np.clip(x, -500, 500)))


def simulate_stream(t=300, drift_strength=0.0, seed=42):
    """Generiert Streaming-Daten."""
    rng = np.random.default_rng(seed)
    # 4 binäre Features
    x = rng.integers(0, 2, size=(t, 4)).astype(float)

    # Basisrate je Phase
    base = np.zeros(t)
    base[:100] = 0.05
    base[100:200] = 0.05 + drift_strength*0.10
    base[200:] = 0.08

    # Feature-Wirkungen je Phase
    betas1 = np.array([0.8,  0.2,  0.1,  0.4])
    betas2 = np.array([0.1,  0.9,  0.5,  0.2])
    betas3 = np.array([0.3,  0.3,  0.6,  0.3])

    def logit_from_rate(p):
        eps=1e-6
        p=np.clip(p, eps, 1-eps)
        return np.log(p/(1-p))

    logit = np.zeros(t)
    for step in range(t):
        if step < 100:
            betas = betas1
            b0 = logit_from_rate(base[step])
        elif step < 200:
            betas = betas2
            b0 = logit_from_rate(base[step])
        else:
            betas = betas3
            b0 = logit_from_rate(base[step])
        logit[step] = b0 + np.dot(betas, x[step])

    true_rate = 1.0/(1.0 + np.exp(-np.clip(logit, -20, 20)))
    y = rng.random(t) < true_rate

    return x.astype(float), y.astype(float), true_rate.astype(float)


def step_update(w, x_t, y_t, y_pred_t, eta=0.1, alpha=0.01):
    """Aktualisiert Gewichte nach Delta-Regel."""
    error = y_t - y_pred_t
    delta_w = eta * error * x_t - alpha * w
    return w + delta_w


def plot_results(y, y_preds, weights_history, true_rate=None):
    """Visualisiert Vorhersagen und Gewichte."""
    T = len(y)
    fig, (ax1, ax2) = plt.subplots(
        2, 1, figsize=(20, 7),
        sharex=True,
        gridspec_kw={'height_ratios': [3, 2]}
    )

    if true_rate is not None:
        ax1.plot(true_rate, 'k--', lw=2, alpha=0.8, label='Wahre Rate (nur Lernzwecke)')

    ax1.scatter(np.arange(T), y, color=PLOT_COLORS.get('neutral', '#7f7f7f'), s=10, alpha=0.35, label='Beobachtungen (y)')
    ax1.plot(y_preds, color=PLOT_COLORS['prediction'], label='Vorhersage (ŷ)', lw=2, alpha=0.8)

    if true_rate is not None:
        ax1.fill_between(np.arange(T), 0, true_rate, alpha=0.08, color=PLOT_COLORS.get('actual', '#ff7f0e'))
    ax1.set_ylabel('Betrugsbasisrate', fontsize=11, fontweight='bold')
    ax1.legend(loc='upper left', fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_title('Betrugserkennung: Adaptive Gewichte mit Delta-Regel', fontsize=12, fontweight='bold')
    ax1.set_ylim(-0.05, 1.05)

    feature_names = ['neues Gerät', 'Nacht', 'Ausland', 'hoher Betrag']
    colors = [PLOT_COLORS['weight'], PLOT_COLORS['accent'], PLOT_COLORS['primary'], PLOT_COLORS['secondary']]
    for i in range(4):
        ax2.plot(weights_history[:, i], label=f'w_{i+1} ({feature_names[i]})', lw=2, color=colors[i])

    ax2.set_ylabel('Gewichte w_i', fontsize=11, fontweight='bold')
    ax2.set_xlabel('Zeit (t)', fontsize=11, fontweight='bold')
    ax2.legend(loc='upper left', ncol=2, fontsize=9)
    ax2.grid(True, alpha=0.3)
    ax2.axhline(0, color='black', linestyle='--', alpha=0.3, lw=1)

    plt.tight_layout()
    plt.show()
    plt.close(fig)

### Interaktives Beispiel: Gewichte-Anpassung

In [3]:
x_data, y_data, true_rate_data = simulate_stream(t=300, drift_strength=0.0)


def update_plot(eta=0.1, alpha=0.01, drift_strength=0.0):
    """Aktualisiert die Plots basierend auf den Parametern."""
    x, y, true_rate = simulate_stream(t=300, drift_strength=drift_strength, seed=SEED)

    w = np.zeros(4)
    y_preds = np.zeros(300)
    weights_history = np.zeros((300, 4))

    for t in range(300):
        y_pred = sigmoid(np.dot(w, x[t]))
        y_preds[t] = y_pred

        w = step_update(w, x[t], y[t], y_pred, eta=eta, alpha=alpha)
        weights_history[t] = w.copy()

    plot_results(y, y_preds, weights_history, true_rate=true_rate)

# Interaktive Slider
interact(
    update_plot,
    eta=widgets.FloatSlider(0.1, min=0.01, max=0.50, step=0.01,
                             description='η (Lernrate):'),
    alpha=widgets.FloatSlider(0.01, min=0.001, max=0.05, step=0.001,
                               description='α (Decay):'),
    drift_strength=widgets.FloatSlider(0.0, min=0.0, max=1.0, step=0.1,
                                        description='Drift-Stärke:')
)

interactive(children=(FloatSlider(value=0.1, description='η (Lernrate):', max=0.5, min=0.01, step=0.01), Float…

<function __main__.update_plot(eta=0.1, alpha=0.01, drift_strength=0.0)>

---

### Interpretation der Ergebnisse

**Was beobachten wir?**

1. **Hohe Lernrate (η = 0.3-0.5):**
   - Schnelle Anpassung an Regimewechsel
   - Aber: Mehr Rauschen, Gewichte schwingen stärker
   - Resultat: Überanpassung möglich

2. **Niedrige Lernrate (η = 0.01-0.05):**
   - Stabile Vorhersage
   - Aber: Langsame Anpassung an Regimewechsel
   - Resultat: Verzögerter Lerneffekt

3. **Hohe Homeostase (α = 0.03-0.05):**
   - Gewichte bleiben klein und stabil
   - Aber: Weniger Plastizität, schwächere Anpassung
   - Resultat: Konservatives Lernen

4. **Niedrige Homeostase (α = 0.001-0.005):**
   - Mehr Plastizität, stärkere Anpassung
   - Aber: Gewichte können unbegrenzt wachsen
   - Resultat: Aggressive Anpassung

5. **Mit Drift (Regimewechsel):**
   - Phase 1 (5% Betrug): System lernt die Basisrate
   - Phase 2 (15% Betrug): System passt sich an (wenn η hoch genug)
   - Phase 3 (8% Betrug): System passt sich wieder an

**Praxisleitfaden:**
- **Konservativ:** Hohe α, niedrige η -> Stabil, aber langsam
- **Aggressiv:** Niedrige α, hohe η -> Schnell, aber rauschig
- **Ausgewogen:** Mittlere Werte -> Balance zwischen Stabilität und Responsivität

---

### Neurobiologische Grundlagen

#### Warum funktioniert dieses Modell biologisch?

**1. Delta-Regel <-> Fehler-getriebenes Lernen**
- Das Gehirn nutzt Fehler-Signale (Dopamin) um zu lernen
- Große Fehler -> Starke Anpassung
- Kleine Fehler -> Schwache Anpassung
- Dies ist biologisch plausibel und experimentell bestätigt

**2. LTP (Langzeitpotenzierung) <-> Verstärkung**
- Wenn präsynaptische Aktivität (x) mit postsynaptischem Fehler-Signal (y - ŷ) koaktiv ist
- Und der Fehler positiv ist (wir unterschätzt haben)
- Dann wird die Synapse stärker (LTP)
- Biologisch: Dopamin-Signal aktiviert Kinasen -> Rezeptoren werden stärker

**3. LTD (Langzeitdepression) <-> Abschwächung**
- Wenn präsynaptische Aktivität mit postsynaptischem Fehler-Signal koaktiv ist
- Und der Fehler negativ ist (wir überschätzt haben)
- Dann wird die Synapse schwächer (LTD)
- Biologisch: Fehler-Signal aktiviert Phosphatasen -> Rezeptoren werden schwächer

**4. Homeostase (−αw) <-> Homeostatic Scaling**
- Der Term −αw verhindert ungebremstes Wachstum
- Biologisch: Rezeptor-Normalisierung, Synapsen-Sättigung vermeiden
- Erhält die Lernfähigkeit des Systems

**5. Koaktivität (x, error) <-> Hebb'sche Regel**
- Nur wenn präsynaptische Aktivität UND postsynaptisches Fehler-Signal gleichzeitig aktiv sind
- Wird die Synapse modifiziert
- Dies ist die biologische Basis der Hebb'schen Regel

---

#### Zusammenfassung
Die Delta-Regel mit Homeostase integriert zentrale neurobiologische Prinzipien – Langzeitpotenzierung und -depression (LTP/LTD), fehlergetriebenes Lernen und homeostatisches Scaling – zu einem kohärenten, anwendungsorientierten Lernmodell.